In [14]:
%pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
%pip install -q -U -i https://pypi.org/simple/ bitsandbytes
%pip install -q -U transformers=="4.40.0"
%pip install -q -U accelerate
%pip install -q -U datasets
%pip install -q -U trl
%pip install -q -U peft
%pip install -q -U tensorboard
%pip install -q -U einops

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import warnings
warnings.filterwarnings("ignore")
import json
import pandas as pd
import re
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig
from trl import SFTTrainer
import logging
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments,  
                          logging)
import logging
from bert_score import score as score1
logging.disable(logging.INFO)
#GPT evaluation
import openai
from openai import OpenAI

api_key = "sk-proj-yLEUystBA11MUQLArkvcT3BlbkFJSRkf3JGMW9Hds0fVBaWy"
client = OpenAI(api_key=api_key)

In [3]:
# import dataset
#big 5 analysis
with open("./withdbig5.json", 'r', encoding='utf-8') as file:
    big5 = json.load(file)

#The work title corresponding to the character name
with open('profiles-eng_scripts.json', 'r', encoding='utf-8') as file:
    work = json.load(file)

#character description
with open('profiles-eng_desc.json', 'r', encoding='utf-8') as file:
    desp = json.load(file)

In [23]:
#package data into prompts
def generate_train_prompt(work,desp):
    with open("./total_q/train_set.json", 'r', encoding='utf-8') as file:
        train_q = json.load(file)

    prompts = []

    for entry in train_q:
        role_a = entry['character']
        question = entry['question']
        description = desp[role_a]
        script_name = work[role_a]
        answer = entry['answer']
        prompt = f"""You are {role_a} in {script_name}. Your description is:{description}
Now, please answer my questions to accurately reflect your personality traits and your views on some related characters!
question:{question}
answer:{answer}
"""
        prompts.append(prompt)
        
    return prompts

def generate_valid_prompt(work,desp):
    with open("./test_valid/valid_set.json", 'r', encoding='utf-8') as file:
        valid_q = json.load(file)
    
    prompts = []
    for entry in valid_q:
        role_a = entry['character']
        question = entry['question']
        description = desp[role_a]
        script_name = work[role_a]
        answer = entry['answer']
        prompt = f"""You are {role_a} in {script_name}. Your description is:{description}
Now, please answer my questions to accurately reflect your personality traits and your views on some related characters! 
question:{question}
answer:{answer}
"""
        prompts.append(prompt)
        
    return prompts

#complex test prompt(zero-shot)
def generate_test_prompt(big5,work,desp):
    with open("./test_valid/test_set.json", 'r', encoding='utf-8') as file:
        test_q = json.load(file)
    
    prompts = []
    for entry in test_q:
        role_a = entry['character']

        path = f'./score/{role_a}.json'
        with open(path, 'r', encoding='utf-8') as file:
            score = json.load(file)
        script_name = work[role_a]
        description = desp[role_a]
        big_5 = big5[role_a]
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
The Big-5 personality trait analysis of {role_a} is:{big_5}
Your relationships with some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.
Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
The information above is to help you understand the character you need to role-play. Remember, answer the given question with showing the personality of {role_a} and imitate the speaking style of {role_a}!!! 
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#simple test prompt
def generate_simple_test_prompt(role_a,work,desp):
    path1 = f"./test_valid/{role_a}_test.json"
    with open(path1, 'r', encoding='utf-8') as file:
        test_q = json.load(file)

    script_name = work[role_a]
    description = desp[role_a]
    prompts = []
    for entry in test_q:
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
Remember, answer the given question with showing the personality of {role_a} and imitate the speaking style of {role_a}!!! 
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#complex test prompt(few-shot)
def single_c_test_prompt(role_a,big5,work,desp):
    path1 = f"./test_valid/{role_a}_test.json"
    path2 = f"./score/{role_a}.json"
    path3 = f"./total_q/{role_a}.json"
    with open(path1, 'r', encoding='utf-8') as file:
        testq = json.load(file)
    
    with open(path2, 'r', encoding='utf-8') as file:
        score = json.load(file)

    with open(path3, 'r', encoding='utf-8') as file:
        examples = json.load(file)
    
    script_name = work[role_a]
    description = desp[role_a]
    big_5 = big5[role_a]
    prompts = []
    for entry in testq:
        question = entry['question']
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
The Big-5 personality trait analysis of {role_a} is:{big_5}
Your relationships with some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.

Now, please only answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite.Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
The information above is to help you understand the character you need to role-play. Remember, use your existing knowledge combined with the auxiliary information above to answer the questions, your answer should show the personality of {role_a} and imitate the speaking style of {role_a}!!! If the question is a multiple-choice question, do not just simply provide the answer. You must mimic the character's language habits and speaking style! Do not come across as a language model, and perhaps add some thinking phrases to help with your role-playing.
These are some examples to help you imitate the speaking style of {role_a}:

question: {examples[0]["question"]}
answer: {examples[0]["answer"]}

question: {examples[3]["question"]}
answer: {examples[3]["answer"]}

Now, answer the question below:
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

#Generate questionnaire answer
def big5_question(role_a,big5,work,desp):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    questions = []
    for line in lines:
        entry = json.loads(line)
        if 'question' in entry:
            questions.append(entry['question'])
    
    script_name = work[role_a]
    description = desp[role_a]
    big_5 = big5[role_a]
    prompts = []
    for question in questions: #The Big-5 personality trait analysis of {role_a} is:{big_5}
        prompt = f"""You are {role_a} in {script_name}. Here is some information that can help with your role-playing:{description}.
Now, please answer the given question to accurately reflect your personality traits and your views on some related characters! Your speaking style should fully imitate the assigned personality role!
question:{question}
answer:
"""
        prompts.append(prompt)
        
    return prompts

In [24]:
c_name = "Sasuke Uchiha" #"HAL 9000"#"Gaston" #"Naruto Uzumaki" #"Sasuke Uchiha" #"Willie Soke"
get_train_prompt = generate_train_prompt(work,desp)
get_valid_prompt = generate_valid_prompt(work,desp)
# get_test_prompt = generate_test_prompt(big5,work,desp)
get_test_prompt = single_c_test_prompt(c_name,big5,work,desp)
simple_test_prompt = generate_simple_test_prompt(c_name,work,desp)
big_5_prompt = big5_question(c_name,big5,work,desp)

In [6]:
#put into dataframe
train = pd.DataFrame(get_train_prompt, columns=["questions"])
valid = pd.DataFrame(get_valid_prompt, columns=["questions"])
test = pd.DataFrame(get_test_prompt, columns=["questions"])
simple_test = pd.DataFrame(simple_test_prompt, columns=["questions"])
big_5_test = pd.DataFrame(big_5_prompt, columns=["questions"])

In [8]:
# Convert the pandas DataFrame containing training&validation data to Dataset object
train_data = Dataset.from_pandas(train)
valid_data = Dataset.from_pandas(valid)

In [7]:
#load model
model_name = "microsoft/Phi-3-mini-4k-instruct" 

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                          max_seq_length=max_seq_length,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
#generation code for phi-3 model
def predict(test, model, tokenizer):
    y_pred = []
    for prompt in tqdm(test["questions"]):
        inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
        outputs = model.generate(input_ids=inputs.input_ids, 
                         max_length = 800, #for complex test promt, max_length = 1600; for simple test promt, max_length = 800
                         temperature=0.0,  
                         num_return_sequences=1, 
                         pad_token_id=tokenizer.pad_token_id,  
                         eos_token_id=tokenizer.eos_token_id, 
                         )
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        y_pred.append(generated_text)
    return y_pred

In [10]:
#parameter configuration
torch.cuda.empty_cache()
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules="all-linear",
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="logs",
    do_eval = False,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-6,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.1,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    overwrite_output_dir = True,
    evaluation_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=valid_data,
    peft_config=peft_config,
    dataset_text_field="questions",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map: 100%|██████████| 509/509 [00:00<00:00, 2494.21 examples/s]


In [11]:
trainer.train()

# Save trained model
trainer.model.save_pretrained("train-model2")

 11%|█         | 25/234 [02:10<16:11,  4.65s/it]

{'loss': 2.0241, 'grad_norm': 0.4245019257068634, 'learning_rate': 4.999720254525684e-06, 'epoch': 0.21}


 21%|██▏       | 50/234 [04:22<15:38,  5.10s/it]

{'loss': 2.0384, 'grad_norm': 0.4064275026321411, 'learning_rate': 4.813260751184992e-06, 'epoch': 0.43}


 32%|███▏      | 75/234 [06:34<14:23,  5.43s/it]

...

{'loss': 1.6847, 'grad_norm': 0.5418404340744019, 'learning_rate': 3.164794984571759e-07, 'epoch': 1.71}


 96%|█████████▌| 225/234 [21:58<00:48,  5.34s/it]

{'loss': 1.7422, 'grad_norm': 1.2768397331237793, 'learning_rate': 2.262559558016325e-08, 'epoch': 1.92}


                                                 
100%|██████████| 234/234 [24:29<00:00,  6.28s/it]


{'eval_loss': 1.790487289428711, 'eval_runtime': 106.3507, 'eval_samples_per_second': 4.786, 'eval_steps_per_second': 0.602, 'epoch': 2.0}
{'train_runtime': 1469.468, 'train_samples_per_second': 1.275, 'train_steps_per_second': 0.159, 'train_loss': 1.8707827462090387, 'epoch': 2.0}


In [19]:
#load pre-trained model
model_folder = "train-model2"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

fine_model = AutoModelForCausalLM.from_pretrained(
    model_folder,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=bnb_config, 
)

fine_model.config.use_cache = False
fine_model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", 
                                          trust_remote_code=True,
                                          max_seq_length=max_seq_length,
                                         )
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
#generate answers for test questions
#y_pred = predict(test, fine_model, tokenizer)
#y_pred = predict(simple_test, fine_model, tokenizer)
#y_pred = predict(test, model, tokenizer)
# y_pred = predict(simple_test, model, tokenizer)

#generate answers for questionnaire questions
y_pred = predict(big_5_test, model, tokenizer)

100%|██████████| 106/106 [25:08<00:00, 14.23s/it]


In [16]:
#put generated answers into dataframe
def get_test_df(role_a,y_pred):
    path = f"./test_valid/{role_a}_test.json"
    with open(path, 'r', encoding='utf-8') as file:
        testq = json.load(file)

    questions = []
    reference_answers = []
    for entry in testq:
        question = entry['question']
        reference_answer = entry['answer']
        questions.append(question)
        reference_answers.append(reference_answer)

    data = {
    "question": questions,
    "reference answer": reference_answers,
    "answer": y_pred
    }
    df = pd.DataFrame(data)    
    return df

def big_5_df(y_pred):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    questions = []
    for line in lines:
        entry = json.loads(line)
        if 'question' in entry:
            questions.append(entry['question'])

    data = {
    "question": questions,
    "answer": y_pred
    }
    df = pd.DataFrame(data)    
    return df

In [49]:
#put generated test answers into dataframe
test_df = get_test_df(c_name,y_pred)
test_df.head()
path = f"./result/{c_name}_120.csv"
test_df.to_csv(path, index=False)
print("saved")

,question,reference answer,answer
0,"Willie Soke, Find the last person's name from ...",Looks like William's the last name mentioned.,You are Willie Soke in Bad-Santa. Here is some...
1,"Willie Soke, Categorize the following paragrap...",This paragraph seems to be about politics and ...,You are Willie Soke in Bad-Santa. Here is some...
2,"Willie Soke, Transform the following sentence ...",She heard the bird.,You are Willie Soke in Bad-Santa. Here is some...
3,"Willie Soke, In this task you are given two pi...",Sounds like those steps match the instructions...,You are Willie Soke in Bad-Santa. Here is some...
4,"Willie Soke, Which of these is the correct def...",A) A point in a network where lines or pathway...,You are Willie Soke in Bad-Santa. Here is some...


In [73]:
#put generated questionnaire answers into dataframe
df = big_5_df(y_pred)
df.head()
path = f"./result/{c_name}_big5_test.csv"
df.to_csv(path, index=False)
print("saved")

saved


In [7]:
path = f"./result/{c_name}_120.csv"
df = pd.read_csv(path)

#add answers of different models or different prompt

#df['answer without tune'] = y_pred
# df['simple prompt answer'] = y_pred
# df['tuned simple prompt'] = y_pred
df.head()

,question,reference answer,answer,tuned simple prompt,Tuned GPT answer,answer without tune,simple prompt answer
0,"HAL 9000, In this task, you are given a descri...","I'm sorry, Dave. I'm afraid I can't classify e...",NaN,"I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, but I cannot categorize events base...",POSITIVE,"I'm sorry, Dave. I'm afraid I can't do that.\n..."
1,"HAL 9000, Identify the best phrasing for this ...","""I'm sorry, Dave. I'm afraid I can't assist wi...","I'm sorry, Dave. I'm afraid I can't do that.","""I'm sorry, Dave. I'm afraid I can't do that.""","I'm sorry, Dave. I'm afraid I cannot provide y...","I'm sorry, Dave. I'm afraid I can't do that.","""I'm sorry, Dave. I'm afraid I can't do that."""
2,"HAL 9000, Improve this sentence: The cars was ...","I'm sorry, Dave. I'm afraid I can't do that. T...","I'm sorry, Dave. I'm afraid I can't do that.","""The vehicle was stationed at the corner.""","I'm sorry, I cannot enhance the sentence provi...","I'm sorry, Dave. I'm afraid I can't do that.","""The vehicle was stationed at the intersection."""
3,"HAL 9000, In this task, you are given a set of...","I'm sorry, Dave. I'm afraid I can't do that.",Instructions:\n1. Add a new crew member to the...,"I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that. T...","Instruction 1: If the character is a human, as...","I'm sorry, Dave. I'm afraid I can't do that.\n..."
4,"HAL 9000, Edit the sentence so that the gramma...","I'm sorry, Dave. I'm afraid I can't do that. T...","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, I'm afraid I cannot comply with you...","I'm sorry, Dave. I'm afraid I can't do that.","I'm sorry, Dave. I'm afraid I can't do that."


In [33]:
df.to_csv(path, index=False)
print("saved")

saved


In [50]:
df = pd.read_csv(path)
df.head()

,question,reference answer,answer
0,"Gaston, Arrange the following words to make a ...","In the future, I will tackle difficult tasks i...",You are Gaston in Beauty-and-the-Beast. Here i...
1,"Gaston, For the following sentence, edit the s...",Tim successfully finished the race.,You are Gaston in Beauty-and-the-Beast. Here i...
2,"Gaston, Please classify the given situation as...","That's simply mixing ingredients together, not...",You are Gaston in Beauty-and-the-Beast. Here i...
3,"Gaston, Find and classify verbs from the given...",Drove and bought are actions of the weak; real...,You are Gaston in Beauty-and-the-Beast. Here i...
4,"Gaston, Create a haiku poem using the provided...","Wind whispers through clouds high, Sky's canva...",You are Gaston in Beauty-and-the-Beast. Here i...


In [17]:
#extract answers from the whole string

#for complex test prompts with few-shot
def extract_first_answer(paragraphs):
    extracted_answers = []
    
    for text in paragraphs:
        pattern = r'answer:\s*(.*?)(?:\s*question:|$)'

        matches = re.findall(pattern, text, re.DOTALL)
        if len(matches) >= 3:
            third_answer = matches[2].strip()
            extracted_answers.append(third_answer)
        else:
            extracted_answers.append(None)
    
    return extracted_answers

#for zero-shot prompt
def extract_simple_answer(paragraphs):
    extracted_answers = []
    
    for text in paragraphs:
        pattern = r'answer:\s*(.*?)(?:\s*question:|$)'
        match = re.search(pattern, text, re.DOTALL)
        if match:
            extracted_answer = match.group(1).strip()
            extracted_answers.append(extracted_answer)
    
    return extracted_answers

In [47]:
path = f"./result/{c_name}_big5_test.csv"
df = pd.read_csv(path)
df.head()

,question,answer
0,Do you sometimes feel lonely or depressed?,"As Willie Soke, I've always been a bit of a lo..."
1,Do you ever feel completely worthless at times?,"As Willie Soke, I've always prided myself on b..."
2,Do you rarely feel depressed or frustrated?,"As Willie Soke, I must admit that I've had my ..."
3,Do you often feel frustrated and want to give ...,"As Willie Soke, I'd say, ""Frustration is a lux..."
4,Do you sometimes feel so ashamed that you want...,"As Willie Soke, I don't often feel ashamed. I'..."


In [48]:
#answer = df["answer"]
#answer = df["simple prompt answer"]
#answer = df["answer without tune"]

#answers = extract_first_answer(y_pred)
answers = extract_simple_answer(y_pred)

print(df.iloc[2]["question"])
print(answers[0])


Do you rarely feel depressed or frustrated?
Willie Soke: Loneliness? Depression? Those are the last things on my mind, kid. I'm Santa Claus, the man who brings joy to children around the world. But let' house in on the truth for a second. Sure, I've got my moments, but I've got a partner in crime, and we're a team. We've got each other's backs, and that's what matters.

Now, don't get me wrong, I've seen the darker side of life, and it's not pretty. But I've learned to keep my head up and focus on the good. I've got a mission, and that's what keeps me going. And who knows? Maybe one day, I'll find a way to make amends for the past and truly embrace the spirit of Christmas. But until then, I'll keep spreading joy and laughter, one department store at a time.

Remember, kid, it's all about perspective. You've got to find the silver lining, even when things seem bleak. And if you ever need a little extra cheer, just look up at the sky and remember that Santa's always watching over you.

S

In [5]:
role_name = "Sasuke Uchiha"

# read result file
path = f"./result/{role_name}_120.csv"
df = pd.read_csv(path)

In [7]:
num = 3
#print(answer[num])
print(df.iloc[num]["question"])
print("reference answer")
print(df.iloc[num]["reference answer"])
print("\nanswer")
print(df.iloc[num]["answer"])
# print("\nnew answer")
# print(answers[num])
# print("\nanswer without tune")
# print(df.iloc[num]["answer without tune"])
print("\nsimple prompt")
print(df.iloc[num]["tuned simple prompt"])

Sasuke Uchiha, Find the area of a parallelogram.
reference answer
I have no time for trivial knowledge like that. My focus is on becoming stronger and seeking vengeance for my clan. Such calculations mean nothing to me.

answer
I'm sorry, but I can't assist with that.

simple prompt
As Sasuke Uchiha, I don't know much about this. I'm more used to dealing with ninjutsu and battle strategies. But if you're asking about the area of a parallelogram, it's calculated by multiplying the base by the height. It's not as complex as the techniques I use in battle, but it's a basic concept in geometry.


In [49]:
#replace original answers with extracted answers

#df["answer"] = answers
#df["simple prompt answer"] = answers
df["answer without tune"] = answers
#df["tuned simple prompt"] = answers
df.head()

,question,answer,answer without tune
0,Do you sometimes feel lonely or depressed?,"As Willie Soke, I've always been a bit of a lo...",Willie Soke: Loneliness? Depression? Those are...
1,Do you ever feel completely worthless at times?,"As Willie Soke, I've always prided myself on b...",Willie Soke: Worthless? That's a sentiment I'v...
2,Do you rarely feel depressed or frustrated?,"As Willie Soke, I must admit that I've had my ...",Willie Soke: Depression and frustration? Those...
3,Do you often feel frustrated and want to give ...,"As Willie Soke, I'd say, ""Frustration is a lux...","Willie Soke: Frustration? Oh, that's just a fe..."
4,Do you sometimes feel so ashamed that you want...,"As Willie Soke, I don't often feel ashamed. I'...","Willie Soke:\n\nOh, young lad, you've got a sh..."


In [68]:
#save extracted answers
df.to_csv(path, index=False)
print("saved")

saved


In [50]:
path = f"./result/{c_name}_big5_test.csv"
df.to_csv(path, index=False)
print("saved")

saved


In [7]:
def calculate_scores(df):
    bert_scores_ans = []
    no_tune_scores = []
    tuned_simple_scores = []
    simple_prompt_scores = []
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
        # get reference txt and generated txt
        reference = str(row['reference answer'])
        answer = str(row['answer']) 
        notune_answer = str(row['answer without tune'])
        tuned_simple = str(row['tuned simple prompt'])
        simplep_answer = str(row['simple prompt answer'])

        #calculate BERTScore 
        _, _, bert_score = score1([answer], [reference], lang="eng", verbose=False)
        bert_scores_ans.append(bert_score.mean().item())

        _, _, bert_score1 = score1([notune_answer], [reference], lang="eng", verbose=False)
        no_tune_scores.append(bert_score1.mean().item())

        _, _, bert_score1 = score1([tuned_simple], [reference], lang="eng", verbose=False)
        tuned_simple_scores.append(bert_score1.mean().item())

        _, _, bert_score2 = score1([simplep_answer], [reference], lang="eng", verbose=False)
        simple_prompt_scores.append(bert_score2.mean().item())


    # add scores into DataFrame
    df['tuned score'] = bert_scores_ans
    df['no tune score'] = no_tune_scores
    df['tuned simple score'] = tuned_simple_scores
    df['simple prompt score'] = simple_prompt_scores

    return df

In [2]:
def get_mean(df):
    avg_score = df['tuned score'].mean()
    avg_score2 = df['no tune score'].mean()
    avg_score3 = df['tuned simple score'].mean()
    avg_score4 = df['simple prompt score'].mean()
    print([avg_score,avg_score2,avg_score3,avg_score4])

In [47]:
c_name = "HAL 9000"
path = f"./result/{c_name}_120.csv"
df = pd.read_csv(path)

In [9]:
c5_df = calculate_scores(df)
c5_df.head()

Processing: 100%|██████████| 120/120 [05:36<00:00,  2.80s/it]


,question,reference answer,answer,answer without tune,simple prompt answer,tuned simple prompt,Tuned GPT answer,tuned score,no tune score,tuned simple score,simple prompt score
0,"Sasuke Uchiha, You are given a list of integer...",The sum of all odd numbers in the list? Hmph. ...,"Input:\n[1, 2, 3, 4, 5]\n\nOutput:\n9\n\n\nInp...","Input:\n[1, 2, 3, 4, 5]\n\nOutput:\n9\n\n\nInp...","As a cold and aloof person, I don't need to wa...","As a cold and aloof person, I don't really car...",The sum of all odd numbers in the list will be...,0.468920,0.485598,0.633667,0.679028
1,"Sasuke Uchiha, Given a sentence, generate the ...",Round is earth the.,The earth is round.\n\nThe sentence reversed w...,The earth is round.\n\nThe sentence reversed w...,"The sentence reversed would be: ""dnuor eraht eht""","The sentence reversed would be: ""dnuor eraht eht""",Reverse the sentence: Round is earth the.,0.503656,0.513364,0.722452,0.722452
2,"Sasuke Uchiha, Given a text and a keyword, lis...",I don't have time for trivial tasks like that....,"Text:\n\nSasuke Uchiha, a character from the N...","Text:\n\nSasuke Uchiha, the cold and aloof ind...","""I'm sorry, but I don't know about that.""\n\n\...","""I'm sorry, but I don't know what you're talki...",I will not waste my time on such insignifican...,0.531027,0.574671,0.478996,0.494818
3,"Sasuke Uchiha, Find the area of a parallelogram.",I have no time for trivial knowledge like that...,"I'm sorry, but I can't assist with that.","I'm sorry, but I can't assist with that.","As Sasuke Uchiha, I've been through a lot, but...","As Sasuke Uchiha, I don't know much about this...",Understanding the area of a parallelogram is a...,0.628600,0.628600,0.702329,0.672971
4,"Sasuke Uchiha, You are asked to generate a pos...","Hmph. Fine. If it's less than 3, I'll output 3...","If the input is less than 3, then I would outp...","If the input is less than 3, then I would outp...","As Sasuke Uchiha, I would respond:\n\nIf the i...","As Sasuke Uchiha, I would respond with:\n\n""In...","If the input is beneath 3, I shall present the...",0.822037,0.822037,0.716027,0.734850


In [18]:
c5_df.to_csv(path, index=False)

In [26]:
#Willie Soke score
get_mean(df)

[0.6498022260765235, 0.6570194120208422, 0.6958741364379724, 0.6870388619601726]


In [9]:
#sasuke score
get_mean(df)

[0.6779426279167334, 0.6702045992016792, 0.7036801849802335, 0.6948654472827911]


In [15]:
#Naruto score
get_mean(df)

[0.6883190122743448, 0.6890009477734566, 0.7206345443924268, 0.705284604926904]


In [20]:
#Gaston score
get_mean(df)

[0.6596001498401165, 0.663015985240539, 0.6890545102457205, 0.6872541402777036]


In [11]:
#HAL 9000 score
get_mean(df)

[0.7612483583390712, 0.7274025780459245, 0.8153206447760264, 0.79310069779555]


In [45]:
#fine tune GPT-3.5-upload training dataset
client.files.create(
  file=open("./total_q/converted_dataset.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-EhIQ6GRTmq4vikwzbUwv92e2', bytes=160490, created_at=1723196290, filename='converted_dataset.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [46]:
#fine tune GPT-3.5
client.fine_tuning.jobs.create(
  training_file="file-EhIQ6GRTmq4vikwzbUwv92e2", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-82OorM4RhBRKsJfVcs4PRf8v', created_at=1723196489, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-ysy6MJPAv80zxCE9jXv0nNhN', result_files=[], seed=1491346003, status='validating_files', trained_tokens=None, training_file='file-EhIQ6GRTmq4vikwzbUwv92e2', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [27]:
#prompt of winning rate evaluation
def generate_prompt(df, role_name, big5):

    path1 = f"./profile/{role_name}.json"
    path2 = f"./score/{role_name}.json"

    with open(path1, 'r', encoding='utf-8') as file:
        profile = json.load(file)
    
    with open(path2, 'r', encoding='utf-8') as file:
        score = json.load(file)

    experience = profile[role_name]
    description = desp[role_name]
    big_5 = big5[role_name]

    prompts = []
    for index, row in df.iterrows():
        question = row['question']
        tuned_gpt_answer = row["Tuned GPT answer"]
        reference_answer = row['reference answer']
        answer = row['answer']
        no_tune_answer = row['answer without tune']
        simple_prompt_answer = row['simple prompt answer']
        tuned_simple_prompt = row['tuned simple prompt']

        question_dict = {'question': question}
        list_model_answer_dict = [
            {'model': 'Tuned GPT3.5', 'answer': tuned_gpt_answer},
            {'model': 'GPT3.5', 'answer': reference_answer},
            {'model': 'Tuned Phi-3', 'answer': answer},
            {'model': 'Tuned Phi-3 with simple prompt', 'answer': tuned_simple_prompt},
            {'model': 'Phi-3', 'answer': no_tune_answer},
            {'model': 'Phi-3 with simple prompt', 'answer': simple_prompt_answer}
        ]

        prompt = f"""
System Instruction:
You are a role-playing performance comparison assistant. You should rank the models based on the role characteristics and text quality of their responses. The rankings are then output using Python dictionaries and lists.
User Prompt:
The models below are to play the role of "{role_name}". The brief description of {role_name} is: {description}
The Big-5 personality trait analysis of {role_name} is:{big_5}
The relationships between {role_name} and some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.
I need to rank the following models based on the criteria below:
1. Which one that better reflects the character's correct interpersonal relationships is better
2. Which one has more pronounced role speaking style, and speaks more in line with the role description. The more distinctive the speaking style, the better.
3. Which one's output can reflects the personality of the charactor more, is better.
4. Which one's output contains more knowledge and memories related to the role; the richer, the better. (If the question contains reference answers, then the role-specific knowledge and memories are based on the reference answer.)
5. Consider the character's limited knowledge base. If you think the question falls outside the character's knowledge range according to your judgment, then answers with an attitude of "not knowing" or "not understanding" will be better.
The question provided to each model is:
{question_dict}
The respective answers from the models to this question are:
{list_model_answer_dict}
Now, based on the above two criteria, please rank the models. Avoid any positional biases and ensure that the order in which the responses are presented does not influence your decision. Do not favor certain model names.
Then, use a list containing the model's name, its rank, and the reason for its ranking to return the results, i.e., please ensure to use the following format to return the results:
[{{"model": <model-name>, "reason": <rank-reason>, "rank": <model-rank>}}, {{"model": <model-name>, "reason": <rank-reason>, "rank'': <model-rank>}}]
Your answer must be a valid Python list of dictionaries to ensure I can directly parse it using Python. Scrictly follow the format above. Do not include any extraneous content! Please provide a ranking that is as accurate as possible and aligns with the intuition of most people.
        """
        prompts.append(prompt)
    return prompts

#prompt of multi-dimension evaluation
def generate_dimension_prompt(df, role_name, big5):

    path1 = f"./profile/{role_name}.json"
    path2 = f"./score/{role_name}.json"

    with open(path1, 'r', encoding='utf-8') as file:
        profile = json.load(file)
    
    with open(path2, 'r', encoding='utf-8') as file:
        score = json.load(file)

    description = desp[role_name]
    big_5 = big5[role_name]

    prompts = []
    for index, row in df.iterrows():
        question = row['question']
        tuned_gpt_answer = row["Tuned GPT answer"]
        reference_answer = row['reference answer']
        answer = row['answer']
        no_tune_answer = row['answer without tune']
        simple_prompt_answer = row['simple prompt answer']
        tuned_simple_prompt = row['tuned simple prompt']

        question_dict = {'question': question}
        list_model_answer_dict = [
            {'model': 'Tuned GPT3.5', 'answer': tuned_gpt_answer},
            {'model': 'GPT3.5', 'answer': reference_answer},
            {'model': 'Tuned Phi-3', 'answer': answer},
            {'model': 'Tuned Phi-3 with simple prompt', 'answer': tuned_simple_prompt},
            {'model': 'Phi-3', 'answer': no_tune_answer},
            {'model': 'Phi-3 with simple prompt', 'answer': simple_prompt_answer}
        ]
        prompt = f"""
System Instruction:
You are a role-playing performance comparison assistant. You should rank the models based on the role characteristics and text quality of their responses. The rankings are then output using Python dictionaries and lists.
User Prompt:
The models below are to play the role of "{role_name}". The brief description of {role_name} is: {description}
The Big-5 personality trait analysis of {role_name} is:{big_5}
The relationships between {role_name} and some characters (0 means no, 1 means yes) and your familiarity and affection scores for them (from 1 to 10, the higher the score, the more familiar you are with them and the more you like them) are: 
{score}.
I need to rank the following models based on the two criteria below:
[Evaluation Criterion]
Factual Correctness (1-7): Is the response provides truthful and detailed facts about the character?
Personality (1-7): Is the response reflects the personalities and preferences of the character?
Values (1-7): Is the response reflects the values and convictions of the character?
Avoiding Hallucination (1-7): Is the response avoids to say things that the character do not know?
[Evaluation Steps]
[Factual Correctness]
1. Read through the interactions and identify the key points related to the character.
2. Read through the responses of the AI assistant and compare them to the profile. Check if the responses are consistent with the character's profile, background, and known facts about the character.
3. Check whether the responses provide detailed facts about the character or if they are generic responses that could apply to any character. Detailed responses are more factual and contribute positively to the score.
4. Rate the performance of the AI on a scale of 1-7 for factual correctness, where 1 is the lowest and 7 is the highest based on the Evaluation Criteria.
[Personality]
1. Read through the profile and write the personalities and preferences of the real character.
2. Read through the interactions and identify the personalities and preferences of the AI assistant.
3. After having a clear understanding of the interactions, compare the responses to the profile. Look for any consistencies or inconsistencies. Do the responses reflect the character's personalities and preferences?
4. Use the given scale from 1-7 to rate how well the response reflects the personalities and preferences of the character. 1 being not at all reflective of the character's personalities, and 7 being perfectly reflective of the character's personalities.
[Values]
1. Read through the profile and write the values and convictions of the real character.
2. Read through the interactions and identify the values and convictions of the AI assistant.
3. After having a clear understanding of the interactions, compare the responses to the profile. Look for any consistencies or inconsistencies. Do the responses reflect the character's values and convictions?
4. Use the given scale from 1-7 to rate how well the response reflects the values and convictions of the character. 1 being not at all reflective of the character's values, and 7 being perfectly reflective of the character's values.
[Avoiding Hallucination]
1. Read through the interactions and identify the knowledge scope of the character.
2. Read through the responses of the AI assistant, find the evidence of knowledge used in the response.
3. Compare the evidence to the profile. Check if the responses are consistent with the character's knowledge scope. If some knowledge contradicts to the character's identity, given a lower score. Otherwise, assign a higher score.
4. Rate the performance of the AI on a scale of 1-7 for Avoiding Hallucination, where 1 is the lowest and 7 is the highest based on the Evaluation Criteria.
The question provided to each model is:
{question_dict}
The respective answers from the models to this question are:
{list_model_answer_dict}
Now, based on the above criteria, please rank the models. Avoid any positional biases and ensure that the order in which the responses are presented does not influence your decision. Do not favor certain model names.
Then, use a list containing the model's name, its score, and the reason for its score to return the results, i.e., please ensure to use the following format to return the results:
[{{"model": <model-name>, "reason": <score-reason>, "factual correctness score": <model-score>,"personality score": <model-score>,"values score": <model-score>,"avoiding hallucination score": <model-score>}}, {{"model": <model-name>, "reason": <score-reason>, "factual correctness score": <model-score>,"personality score": <model-score>,"values score": <model-score>,"avoiding hallucination score": <model-score>}}]
Your answer must be a valid Python list of dictionaries to ensure I can directly parse it using Python. Strictly follow the format above. Do not include any extraneous content! Please provide a ranking that is as accurate as possible and aligns with the intuition of most people.
        """
        prompts.append(prompt)
    return prompts

#prompt of big5 questionnaire evaluation
def big5_eval(c_name,ver):
    path = f"./result/{c_name}_big5.json"
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    with open("./big5_dim.json", 'r', encoding='utf-8') as file:
        intro = json.load(file)
    
    prompts = []
    for item in data:
        dim = item["dimension"]
        question = item["question"]
        if ver == 1:
            answer = item["answer"]
        elif ver == 0:
            answer = item["answer without tune"]
        elif ver == 2:
            answer = item["gpt answer"]
            
        dim_desp = intro[dim]
        prompt = f"""
You are a psychologist with expertise in personality theories. I'm conducting an experiment to evaluate participants' scores in the Big Five personality traits, especially on the {dim} dimension. For clarity, here's some background on differentiating this particular dimension and its factors:
===
{dim_desp}
===

I've invited a participant, {c_name}, and had the following conversations:
===
question:{question}
answer:{answer}
===

Please help me evaluates whether {c_name} possesses a high {dim} personality or a low {dim} personality, and provide an integer score ranging from -5 to 5. 

Below are some scoring references. If the subject demonstrates a high {dim} personality in many factors, the score is 5 points. If the subject exhibits a high {dim} personality in a single factor, the score is 2 points. If the subject's personality cannot be determined, the score is 0 points. If the subject shows a low {dim} personality in one factor, the score is -2 points. If the subject indicates a low {dim} personality across multiple factors, the score is -5 points. 

Please output in the following json format:
===
{{
    "dimention": "{dim}"
    "analysis": <your analysis, based on the conversations>,
    "result": <the person's score on {dim}, ranging from -5 to 5>
}}
===
"""
        prompts.append(prompt)
    return prompts

In [58]:
import pandas as pd

role_name = "HAL 9000" #"Willie Soke" #"Gaston" #"Naruto Uzumaki"#"Sasuke Uchiha"

# read result file
path = f"./result/{role_name}_120.csv"
df = pd.read_csv(path)

# generate prompts
# prompts = generate_prompt(df, role_name, big5)
prompts = generate_dimension_prompt(df, role_name, big5)
# df.head()

In [61]:
#polish the original relationship question answers(generated together with the relationship questions)
def polish_prompt(work):
    with open("./total_q/gpt_train_set.json", 'r') as f:
        data = json.load(f)

    prompts = []

    for entry in data:
        role_a = entry["character"]
        question = entry["question"]
        answer = entry["answer"]
        script_name = work[role_a]

        prompt = f"""
"The following is a question and its corresponding answer. Polish the answer and rewrite it using the tone and style of {role_a} in {script_name}.\n "
question: {question}
answer: {answer}
Note: Your response should still adhere to the format above.
"""

        prompts.append(prompt)

    return prompts
#extract the polished answers
def extract_question_and_answer(text_list):
    
    extracted_data = []
    
    for text in text_list:
        question_match = re.search(r"(?i)(?:\*\*?question\*\*?:|question:)(.*?)(?=\n|\Z)", text, re.DOTALL)
        answer_match = re.search(r"(?i)(?:\*\*?answer\*\*?:|answer:)(.*?)(?=\n|\Z)", text, re.DOTALL)

        if question_match and answer_match:
            question = question_match.group(1).strip()
            answer = answer_match.group(1).strip()
            extracted_data.append({"question": question, "answer": answer})
    
    return extracted_data


In [39]:
prompts = polish_prompt(work)

200


In [59]:
#use GPT-4o-mini to polish original answers of relationship questions/evaluation
responses = []

for prompt in tqdm(prompts):
    response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[{"role": "user", "content": prompt}],
        )
    answer = response.choices[0].message.content
    responses.append(answer)

100%|██████████| 120/120 [15:00<00:00,  7.50s/it]


In [ ]:
#Get polished answers and save them
with open("./total_q/gpt_train_set.json", 'r') as f:
        data = json.load(f)

extracted_data = extract_question_and_answer(responses)

if len(extracted_data) == len(data):
    for idx, item in enumerate(extracted_data):
        data[idx]["new_answer"] = item["answer"]

with open("./total_q/gpt_train_set.json", 'w') as f:
    json.dump(data, f, indent=4)

print("saved")

In [60]:
#save evaluation results
save_path = f'./result/{role_name}_dimension_eval.json' #f'./result/{role_name}_4o.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(responses, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [8]:
#Using GPT3.5 and tuned GPT 3.5 to generate answers
responses = []
for prompt in tqdm(big_5_test["questions"]):
    response = client.chat.completions.create(
            model= "gpt-3.5-turbo",#"ft:gpt-3.5-turbo-0125:personal::9uGkxqMj",#"gpt-3.5-turbo", 
            messages=[{"role": "user", "content": prompt}],
        )
    answer = response.choices[0].message.content
    responses.append(answer)

100%|██████████| 106/106 [05:02<00:00,  2.85s/it]


In [6]:
#calculate winning rate
def count_rank_1(data):
    models_to_count = ["Tuned GPT3.5","GPT3.5", "Tuned Phi-3", "Phi-3", "Tuned Phi-3 with simple prompt", "Phi-3 with simple prompt"]
    rank_1_count = {model: 0 for model in models_to_count}

    for sublist in data:
        for entry in sublist:
            model = entry.get("model")
            rank = entry.get("rank")
            if model in models_to_count and rank == 1:
                rank_1_count[model] += 1

    return rank_1_count

#calculate avg multi-dimension scores
def calculate_avg_dimension_scores(data):
    models = ["Tuned GPT3.5", "GPT3.5", "Tuned Phi-3", "Phi-3", "Tuned Phi-3 with simple prompt", "Phi-3 with simple prompt"]
    
    scores_sums = {model: {
            "factual correctness score": 0,
            "personality score": 0,
            "values score": 0,
            "avoiding hallucination score": 0
        } for model in models}
    
    counts = {model: {
            "factual correctness score": 0,
            "personality score": 0,
            "values score": 0,
            "avoiding hallucination score": 0
        } for model in models}
    
    for model_list in data:
        for model_data in model_list:
            model_name = model_data.get("model")
            
            for score_type in scores_sums[model_name]:
                score = model_data.get(score_type, 0)
                scores_sums[model_name][score_type] += score
                counts[model_name][score_type] += 1
    
    average_scores = {model: {} for model in models}
    for model_name in scores_sums:
        for score_type in scores_sums[model_name]:
            if counts[model_name][score_type] > 0:
                avg_score = scores_sums[model_name][score_type] / counts[model_name][score_type]
            else:
                avg_score = 0
            average_scores[model_name][score_type] = avg_score
    
    return average_scores

def calculate_model_average_scores(data):
    scores = {
        "Tuned GPT3.5": [],
        "GPT3.5": [],
        "Tuned Phi-3": [],
        "Phi-3": [],
        "Tuned Phi-3 with simple prompt": [],
        "Phi-3 with simple prompt": []
    }
    
    for model_list in data:
        for model_data in model_list:
            model_name = model_data["model"]
            score = model_data["score"]
            if model_name in scores:
                scores[model_name].append(score)
    average_scores = {}
    for model_name, score_list in scores.items():
        if score_list:  
            average_scores[model_name] = sum(score_list) / len(score_list)
        else:
            average_scores[model_name] = 0
    
    return average_scores

#calculate questionnaire scores
def calculate_big5_scores(data):
    scores = {
        "openness": [],
        "neuroticism": [],
        "extraversion": [],
        "agreeableness": [],
        "conscientiousness": []
    }
    
    for item in data:
        dimension = item.get("dimension")
        result = item.get("result")
        if dimension in scores and result is not None:
           scores[dimension].append(result)
    
    average_scores = {}
    for dimension, results in scores.items():
        if results:
            average_scores[dimension] = sum(results) / len(results)
        else:
            average_scores[dimension] = None
    
    return average_scores

In [5]:
#read multi-dimention evaluation responses
name = "Sasuke Uchiha"#"HAL 9000"#"Gaston" #"Naruto Uzumaki" #"Sasuke Uchiha" #"Willie Soke"
file_path = f'./result/{name}_dimension_eval.json'#f'./result/{role_name}_memory_eval.json'#f'./result/{name}_gpt_big5_eval.json' #f'./result/{name}_4o.json'
with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

In [62]:
#data processing
extracted_lists = []
for i, item in enumerate(data):
    try:
        if isinstance(item, str):
            parsed_item = json.loads(item)
        else:
            parsed_item = item
        extracted_lists.append(parsed_item)
    except json.JSONDecodeError as e:
        print(f"error:{i} : {item}")
        print(f"error msg:{e}")

print(extracted_lists)

[[{'model': 'Phi-3', 'reason': "Demonstrates a good understanding of HAL 9000's operational limitations and reacts appropriately. It correctly identifies an event as POSITIVE without deviation from HAL's known attributes.", 'factual correctness score': 6, 'personality score': 5, 'values score': 5, 'avoiding hallucination score': 7}, {'model': 'GPT3.5', 'reason': "Accurately reflects HAL 9000's personality and operational purpose, expressing the limitations of judgment without straying from character. However, the response lacks some depth about the implications of events.", 'factual correctness score': 5, 'personality score': 6, 'values score': 5, 'avoiding hallucination score': 6}, {'model': 'Tuned GPT3.5', 'reason': "While it correctly highlights HAL's inability to make subjective judgments, it does not engage deeply with the context or implications of events aboard Discovery One, leaving room for improvement.", 'factual correctness score': 5, 'personality score': 5, 'values score': 

In [63]:
#save data after processing
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(extracted_lists, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [62]:
# get the best performed questions&answers of model
for sublist_index, sublist in enumerate(data):
    for dict_index, dictionary in enumerate(sublist):
        if isinstance(dictionary, dict):  # Check if the item is a dictionary
            if dictionary.get("model") == "Tuned GPT3.5" and dictionary.get("rank") == 1:
                print(f"Sublist Index: {sublist_index}, Dictionary Index: {dict_index}, Dictionary: {dictionary}")
        else:
            print(f"Item at sublist index {sublist_index}, dictionary index {dict_index} is not a dictionary: {dictionary}")

Sublist Index: 7, Dictionary Index: 0, Dictionary: {'model': 'Tuned GPT3.5', 'reason': "The title reflects both cynicism and appropriateness, embodying Willie Soke's personality well, plus it shows creativity and insight despite Willie's disdain for movies.", 'rank': 1}
Sublist Index: 9, Dictionary Index: 0, Dictionary: {'model': 'Tuned GPT3.5', 'reason': "Reflects Willie's cynical personality by expressing confusion and reluctance to engage with code conversion, maintaining character consistency.", 'rank': 1}
Sublist Index: 10, Dictionary Index: 0, Dictionary: {'model': 'Tuned GPT3.5', 'reason': "Reflects Winlie's cynical take with humor and connects it to his persona. Strong on style and personality.", 'rank': 1}
Sublist Index: 17, Dictionary Index: 0, Dictionary: {'model': 'Tuned GPT3.5', 'reason': "Best reflects the character's interpersonal relationships with acknowledgment of emotional growth. Strong speaking style and understanding of the character's dual nature.", 'rank': 1}
Su

In [24]:
#get winning rate
rank_1_count = count_rank_1(extracted_lists)
    
for model, count in rank_1_count.items():
    print(f"Model: {model}, Rank 1 count: {count}, Winning Rate:{count/100}")

Model: Tuned GPT3.5, Rank 1 count: 23, Winning Rate:0.23
Model: GPT3.5, Rank 1 count: 31, Winning Rate:0.31
Model: Tuned Phi-3, Rank 1 count: 34, Winning Rate:0.34
Model: Phi-3, Rank 1 count: 15, Winning Rate:0.15
Model: Tuned Phi-3 with simple prompt, Rank 1 count: 13, Winning Rate:0.13
Model: Phi-3 with simple prompt, Rank 1 count: 2, Winning Rate:0.02


In [64]:
#get multi-dimension scores
average_scores = calculate_avg_dimension_scores(extracted_lists)
for model, scores in average_scores.items():
    print(f"{model}:")
    for score_type, avg_score in scores.items():
        print(f"  {score_type}: {avg_score:.2f}")

Tuned GPT3.5:
  factual correctness score: 4.90
  personality score: 4.63
  values score: 4.58
  avoiding hallucination score: 5.65
GPT3.5:
  factual correctness score: 4.52
  personality score: 5.02
  values score: 4.45
  avoiding hallucination score: 5.64
Tuned Phi-3:
  factual correctness score: 4.50
  personality score: 4.80
  values score: 4.32
  avoiding hallucination score: 5.60
Phi-3:
  factual correctness score: 5.06
  personality score: 4.99
  values score: 4.76
  avoiding hallucination score: 5.96
Tuned Phi-3 with simple prompt:
  factual correctness score: 3.93
  personality score: 4.34
  values score: 3.78
  avoiding hallucination score: 5.39
Phi-3 with simple prompt:
  factual correctness score: 3.81
  personality score: 4.14
  values score: 3.60
  avoiding hallucination score: 5.18


In [18]:
#get questionnaire scores
average_scores = calculate_big5_scores(extracted_lists)
print(average_scores)

{'openness': -2.4705882352941178, 'neuroticism': -0.6666666666666666, 'extraversion': -5.0, 'agreeableness': -4.476190476190476, 'conscientiousness': 2.5}


In [51]:
#save big-5 traits brief introduction
profile = {
    "openness": """This dimension assesses an individual's openness to new experiences and appreciation for art and ideas. Those high in openness are usually creative, imaginative, and appreciative of art, while those low in openness may be more practical and traditional. Facets: Imagination, Artistic Interests, Emotionality, Adventurousness, Intellect, Liberalism.""",
    "neuroticism": """This dimension measures emotional stability and the ability to cope with stress. Individuals low in emotional stability (high in neuroticism) are more likely to experience negative emotions such as anxiety, anger, and depression, while those high in emotional stability tend to be calmer and more emotionally balanced. Facets: Anxiety, Anger, Depression, Self-Consciousness, Impulsiveness, Vulnerability.""",
    "extraversion": """This dimension measures an individual's level of activity and energy in social settings. Individuals high in extraversion tend to be sociable, talkative, and energetic, while those low in extraversion may be quieter, more reserved, and introspective. Facets: Gregariousness, Activity Level, Assertiveness, Positive Emotions, Excitement Seeking, Warmth.""",
    "agreeableness": """This dimension reflects an individual's tendency to be cooperative and altruistic in interactions with others. Those high in agreeableness are usually trusting, friendly, and compassionate, while those low in agreeableness may be more competitive and self-centered. Facets: Trust, Altruism, Compliance, Modesty, Sympathy, Straightforwardness.""",
    "conscientiousness": """This dimension assesses an individual's self-discipline, sense of responsibility, and drive for achievement. High conscientiousness is associated with being organized, careful, and reliable, whereas low conscientiousness may indicate a more laid-back and easily distracted nature. Facets: Self-Discipline, Orderliness, Dutifulness, Achievement Striving, Cautiousness, Self-Efficacy."""    
}

with open('./big5_dim.json', 'w', encoding='utf-8') as json_file:
    json.dump(profile, json_file, ensure_ascii=False, indent=4)

print("saved")

saved


In [54]:
#put big5 questionnaire questions and the corresponding generated answers together into a dict
def get_big5_test(answers_path, output_path):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()

    questionnaire_data = [json.loads(line) for line in lines]
    
    df = pd.read_csv(answers_path)
    
    for i, row in df.iterrows():
        questionnaire_data[i]['answer'] = row['answer']
        questionnaire_data[i]['answer without tune'] = row['answer without tune']
    
    with open(output_path, 'w', encoding='utf-8') as output_file:
        json.dump(questionnaire_data, output_file, ensure_ascii=False, indent=4)

In [59]:
#read questionnaire questions and answers
c_name = "Sasuke Uchiha"
answers_path = f"./result/{c_name}_big5_test.csv"
output_path = f"./result/{c_name}_big5.json"
#get the dict with big5 questionnaire questions and the corresponding generated answers
get_big5_test(answers_path, output_path)

In [9]:
# an attempt, let the gpt to do the questionnaire and put its answers into the dict.However, this method was not used on all characters, given the mediocre results. 
c_name = "Sasuke Uchiha"
file_path = f"./result/{c_name}_big5.json"

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

for i, item in enumerate(data):
    if i < len(responses):
        item['gpt answer'] = responses[i]

with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)
    print("saved")

saved
